In [235]:
import os 
import pandas as pd
import numpy as np

In [236]:
data_path = '../data/2023-07-19'
likes_path = os.path.join(data_path, '_like__202307201554.csv')
outfit_path = os.path.join(data_path, 'outfit_202307201554.csv')

likes = pd.read_csv(likes_path)
session = pd.read_csv('../data/_session__202307201554.csv')
outfit = pd.read_csv(outfit_path)
similar = pd.read_csv('../data/_similar__202307201554.csv')
user =pd.read_csv('../data/_user__202307201554.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/2023-07-19/_like__202307201554.csv'

# Data Base: postgres
- likes<br>
좋아요 클릭
- similar<br>
유사한 아이템 목록
- user<br>
회원 정보
- outfit<br>
옷 메타 정보

In [237]:
likes.head(5)

,like_id,session_id,user_id,outfit_id,timestamp,like_type,is_deleted,as_login
0,1,66178898-dfb6-4de4-8f2a-cb09e3ccbe11,NaN,73016,2023-07-18 23:53:25.086,journey,False,False
1,2,66178898-dfb6-4de4-8f2a-cb09e3ccbe11,NaN,79507,2023-07-18 23:53:25.715,journey,False,False
2,3,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,88572,2023-07-18 23:53:53.555,journey,False,False
3,4,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,88731,2023-07-18 23:53:54.075,journey,False,False
4,5,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,78346,2023-07-18 23:53:55.314,journey,False,False


In [238]:
likes['session_id'].nunique()

179

## Side Information

In [124]:
outfit.head(2)

,outfit_id,img_url,gender,age,origin_url,reporter,tags,brands,region,occupation,style,date
0,87435,https://codidatabucket.s3.ap-northeast-2.amazo...,F,21.0,https://www.musinsa.com/mz/streetsnap/view/87435,라이트 그레이 컬러의 엠엘비 청키 라이너 스니커즈를 매칭한 룩입니다.,"{겨울,캐주얼,여성,스니커즈,엠엘비}",{엠엘비},신사/압구정,학생,캐주얼,2022-12-30 00:00:00.000
1,87434,https://codidatabucket.s3.ap-northeast-2.amazo...,F,22.0,https://www.musinsa.com/mz/streetsnap/view/87434,화이트 컬러의 엠엘를 청키 라이너 스니커즈를 매칭한 룩입니다.,"{겨울,캐주얼,여성,스니커즈,엠엘비}",{엠엘비},신사/압구정,전문직/프리랜서,캐주얼,2022-12-29 00:00:00.000


In [125]:
#columns = ['outfit_id', 'img_url', 'gender', 'age', 'origin_url', 'reporter', 'tags', 'brands', 'region', 'occupation', 'style', 'date']

selected_columns = ['outfit_id', 'gender', 'age', 'tags', 'brands', 'style', 'date']
selected_outfit = outfit[selected_columns]
selected_outfit.rename(columns={'date': 'outfit_date'}, inplace=True)

/opt/conda/envs/jun/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [126]:
selected_outfit['tags'] = selected_outfit['tags'].apply(lambda x: [tag.strip('"') for tag in x.strip('{}').split(',')])
selected_outfit['brands'] = selected_outfit['brands'].fillna('').apply(lambda x: [brand.strip('"') for brand in x.strip('{}').split(',')])

<ipython-input-126-6ec147a53de5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_outfit['tags'] = selected_outfit['tags'].apply(lambda x: [tag.strip('"') for tag in x.strip('{}').split(',')])
<ipython-input-126-6ec147a53de5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_outfit['brands'] = selected_outfit['brands'].fillna('').apply(lambda x: [brand.strip('"') for brand in x.strip('{}').split(',')])


In [127]:
selected_outfit['outfit_date'] = pd.to_datetime(selected_outfit['outfit_date']).dt.date

<ipython-input-127-21b5e0e0a62c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_outfit['outfit_date'] = pd.to_datetime(selected_outfit['outfit_date']).dt.date


In [128]:
selected_outfit

,outfit_id,gender,age,tags,brands,style,outfit_date
0,87435,F,21.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-30
1,87434,F,22.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
2,87433,F,30.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
3,87432,F,21.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[],캐주얼,2022-12-29
4,87428,F,22.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
...,...,...,...,...,...,...,...
10424,71873,M,NaN,"[남성, 반팔 티셔츠, 데님 팬츠, 스니커즈, 발렌시아가, 여름, 스트릿]",[발렌시아가],스트릿,2021-06-14
10425,71801,M,NaN,"[남성, 반팔 셔츠, 치노 팬츠, 샌들, 여름, 캐주얼]",[],캐주얼,2021-06-09
10426,73922,M,29.0,"[유니클로, 컨버스, 여름, 스트릿]",[컨버스],스트릿,2021-08-23
10427,72586,M,NaN,"[미니, 크로스 백, 컨버스, 데우스 엑스 마키나, 여름, 스트릿]","[컨버스, 데우스 엑스 마키나]",스트릿,2021-07-06


In [129]:
# 'tags' 열에서 '봄'이라는 값을 포함하는 행들만 뽑아내기
spring_outfit = selected_outfit[selected_outfit['tags'].apply(lambda x: '봄' in x)]
summer_outfit = selected_outfit[selected_outfit['tags'].apply(lambda x: '여름' in x)]
fall_outfit = selected_outfit[selected_outfit['tags'].apply(lambda x: '가을' in x)]
winter_outfit = selected_outfit[selected_outfit['tags'].apply(lambda x: '겨울' in x)]

In [130]:
summer_outfit

,outfit_id,gender,age,tags,brands,style,outfit_date
1459,84295,F,20.0,"[여성, 밀리터리, 카고팬츠, 여름]",[],캐주얼,2022-08-31
1478,84294,F,20.0,"[여성, 데님, 스커트, 티셔츠, 여름]",[],캐주얼,2022-08-30
1483,84314,F,22.0,"[여성, 튜브탑, 데님팬츠, 크로스백, 여름]",[],캐주얼,2022-08-31
1484,84313,F,27.0,"[여성, 카디건, 반소매, 데님팬츠, 여름]",[],걸리시,2022-08-31
1485,84312,F,24.0,"[여성, 슬리브리스, 크롭셔츠, 체크팬츠, 여름]",[],캐주얼,2022-08-31
...,...,...,...,...,...,...,...
10424,71873,M,NaN,"[남성, 반팔 티셔츠, 데님 팬츠, 스니커즈, 발렌시아가, 여름, 스트릿]",[발렌시아가],스트릿,2021-06-14
10425,71801,M,NaN,"[남성, 반팔 셔츠, 치노 팬츠, 샌들, 여름, 캐주얼]",[],캐주얼,2021-06-09
10426,73922,M,29.0,"[유니클로, 컨버스, 여름, 스트릿]",[컨버스],스트릿,2021-08-23
10427,72586,M,NaN,"[미니, 크로스 백, 컨버스, 데우스 엑스 마키나, 여름, 스트릿]","[컨버스, 데우스 엑스 마키나]",스트릿,2021-07-06


In [105]:
fall_outfit

,outfit_id,gender,age,tags,brands,style,outfit_date
187,87130,F,19.0,"[가을, 시크]",[],시크,2022-12-19
273,86832,F,20.0,"[가을, 캐주얼]",[],캐주얼,2022-12-06
288,86861,F,24.0,"[가을, 걸리시]",[],걸리시,2022-12-07
289,86859,F,22.0,"[가을, 캐주얼]",[],캐주얼,2022-12-07
317,86918,F,22.0,"[가을, 걸리시]",[],걸리시,2022-12-08
...,...,...,...,...,...,...,...
7985,84354,M,24.0,"[남성, 롱슬리브, 트레이닝팬츠, 발렌시아가, 보테가베네타, 카세트백, 가을]",[],스트릿,2022-09-01
7986,84344,M,23.0,"[남성, 슬리브리스, 트러커재킷, 데님팬츠, 가을]",[],캐주얼,2022-09-01
7987,84340,M,25.0,"[남성, 롱슬리브, 데님팬츠, 그린, 볼캡, 가을]",[],댄디,2022-09-01
7988,84325,M,26.0,"[남성, 체커보드, 패턴, 크로스백, 가을]",[],캐주얼,2022-09-01


In [119]:
selected_outfit['brands']

0                    [엠엘비]
1                    [엠엘비]
2                    [엠엘비]
3                       []
4                    [엠엘비]
               ...        
10424              [발렌시아가]
10425                   []
10426                [컨버스]
10427    [컨버스, 데우스 엑스 마키나]
10428                   []
Name: brands, Length: 10429, dtype: object

In [113]:
selected_outfit

,outfit_id,gender,age,tags,brands,style,outfit_date
0,87435,F,21.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-30
1,87434,F,22.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
2,87433,F,30.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
3,87432,F,21.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[],캐주얼,2022-12-29
4,87428,F,22.0,"[겨울, 캐주얼, 여성, 스니커즈, 엠엘비]",[엠엘비],캐주얼,2022-12-29
...,...,...,...,...,...,...,...
10424,71873,M,NaN,"[남성, 반팔 티셔츠, 데님 팬츠, 스니커즈, 발렌시아가, 여름, 스트릿]",[발렌시아가],스트릿,2021-06-14
10425,71801,M,NaN,"[남성, 반팔 셔츠, 치노 팬츠, 샌들, 여름, 캐주얼]",[],캐주얼,2021-06-09
10426,73922,M,29.0,"[유니클로, 컨버스, 여름, 스트릿]",[컨버스],스트릿,2021-08-23
10427,72586,M,NaN,"[미니, 크로스 백, 컨버스, 데우스 엑스 마키나, 여름, 스트릿]","[컨버스, 데우스 엑스 마키나]",스트릿,2021-07-06


# 클릭


In [167]:
import os

def txt_to_csv(txt_file_path, csv_file_path):
    with open(txt_file_path, 'r') as txt_file:
        with open(csv_file_path, 'w') as csv_file:
            for line in txt_file:
                csv_file.write(line.replace('\n', '') + '\n')

def process_files(data_path, file_names):
    for file_name in file_names:
        input_txt_path = os.path.join(data_path, f"{file_name}.txt")
        output_csv_path = os.path.join(data_path, f"{file_name}.csv")
        txt_to_csv(input_txt_path, output_csv_path)

# 데이터 폴더 경로 설정
data_path = "/opt/ml/input/styl-ml/data/2023-07-19"

# 처리할 파일 이름 리스트 설정
file_names = ["click_image_log", "click_share_musinsa_log", "view_image_log"]

process_files(data_path, file_names)

In [168]:
click_image_path = os.path.join(data_path, 'click_image_log.csv')

In [169]:
click_image = pd.read_csv(click_image_path, sep=',')

In [170]:
click_image

,session_id,user_id,outfit_id,timestamp,click_type
0,7708c8e7-4292-4ff9-99b1-27be20427e42,1,83783,23-07-19 09:26:07,collection
1,7708c8e7-4292-4ff9-99b1-27be20427e42,1,83800,23-07-19 09:26:10,similar
2,7708c8e7-4292-4ff9-99b1-27be20427e42,1,83791,23-07-19 09:26:12,similar
3,7708c8e7-4292-4ff9-99b1-27be20427e42,1,84029,23-07-19 09:26:16,similar
4,7708c8e7-4292-4ff9-99b1-27be20427e42,1,83706,23-07-19 09:26:18,similar
...,...,...,...,...,...
1101,4a6fbc97-f978-4746-93e9-c397d6b7551a,51,81583,23-07-20 07:33:34,similar
1102,4a6fbc97-f978-4746-93e9-c397d6b7551a,51,90772,23-07-20 07:33:42,journey
1103,4a6fbc97-f978-4746-93e9-c397d6b7551a,51,91240,23-07-20 07:33:44,similar
1104,4a6fbc97-f978-4746-93e9-c397d6b7551a,51,91744,23-07-20 07:33:45,similar


In [40]:
click_image['session_id'].nunique()

129